In [1]:
import pandas as pd 
import numpy as np

In [27]:
books = pd.read_csv(r"C:\Users\DEEPANSHU NARWAT\notebooks\book-recomm\BX-Books.csv", sep=';', encoding="latin-1", error_bad_lines=False)
users = pd.read_csv(r"C:\Users\DEEPANSHU NARWAT\notebooks\book-recomm\BX-Users.csv", sep=';', encoding="latin-1", error_bad_lines=False)
ratings = pd.read_csv(r"C:\Users\DEEPANSHU NARWAT\notebooks\book-recomm\BX-Book-Ratings.csv", sep=';', encoding="latin-1", error_bad_lines=False)

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.p

In [28]:
books = books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher']]
books.rename(columns={'Book-Title':'title' , 'Book-Author':'author' , 'Year-Of-Publication':'year' , 'Publisher':'publisher'}, inplace=True)

In [29]:
users.rename(columns={'User-ID':'userid' , 'Location':'location' , 'Age':'age'}, inplace=True)

In [30]:
ratings.rename(columns={'User-ID':'userid' , 'Book-Rating':'rating'}, inplace=True)

In [33]:
x = ratings['userid'].value_counts() > 200

In [34]:
y = x[x].index
print(y.shape)

(899,)


In [35]:
ratings = ratings[ratings['userid'].isin(y)]

In [52]:
ratings_with_books = ratings.merge(books , on='ISBN')
ratings_with_books.head()

,userid,ISBN,rating,title,author,year,publisher
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc


In [71]:
ratings_with_books.drop('number_of_ratings', axis=1, inplace=True)

In [74]:
number_rating = ratings_with_books.groupby('title')['rating'].count().reset_index()

In [76]:
number_rating.rename(columns={'rating':'number_of_rating'}, inplace=True)

In [77]:
final_rating = ratings_with_books.merge(number_rating, on='title')

In [81]:
final_rating.drop_duplicates(['userid', 'title'], inplace=True)

In [82]:
final_rating.shape

(483423, 8)

In [83]:
final_rating = final_rating[final_rating['number_of_rating'] > 50]

In [84]:
final_rating.shape

(57952, 8)

In [85]:
book_pivot = final_rating.pivot_table(columns='userid', index='title', values='rating')
book_pivot.fillna(0, inplace=True)

In [88]:
book_pivot.shape

(703, 888)

In [89]:
from scipy.sparse import csr_matrix
book_sparse = csr_matrix(book_pivot)

In [90]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm='brute')
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [102]:
distance, suggestions = model.kneighbors(book_pivot.iloc[237, :].values.reshape(1, -1))

In [103]:
for i in range(len(suggestions)):
    print(book_pivot.index[suggestions[i]])

Index(['Hemlock Bay', 'Exclusive', 'Jacob Have I Loved', 'No Safe Place',
       'The Cradle Will Fall'],
      dtype='object', name='title')


In [105]:
model.kneighbors_graph()

<703x703 sparse matrix of type '<class 'numpy.float64'>'
	with 3515 stored elements in Compressed Sparse Row format>